## 2. Limpieza de los datos de los Perfiles de Ingreso

En esta libreta se realiza la limpieza de los datos del perfil de ingreso de alumnos cuya inscripción a la Universidad de Sonora ocurrió en los años 2014, 2015, 2016 y 2017. Únicamente se tomaron en cuenta estos años debido a que el último reporte histórico que se está considerando es el del 2022-1, justo cuando los estudiantes del 2017 comienzan a ser alumnos rezagados.

Primero, se leen todos los archivos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Límites de renglones y columnas a mostrar al imprimir DFs
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

In [2]:
anios = [2014, 2015, 2016, 2017]
dfPerfIn = {}

for a in anios:
    df = pd.read_pickle(f'../dfsWithCorrectedSchema/perfilIngreso/{a}.pkl')
    dfPerfIn[a] = df

Se revisa el número de registros iniciales

In [3]:
print('----- NUM. DE REGISTROS INICIALES EN LOS DATOS DEL PERFIL DE INGRESO -----\n')
# Se almacenan los dataframes en una carpeta local
totalRen = 0 
for anio in dfPerfIn.keys():
    rengAnio = len(dfPerfIn[anio])
    totalRen = totalRen + rengAnio
    print(f'Num. de registros en el dataframe del {anio}: {rengAnio}', )
print(f'Núm. registros totales: {totalRen}')

----- NUM. DE REGISTROS INICIALES EN LOS DATOS DEL PERFIL DE INGRESO -----

Num. de registros en el dataframe del 2014: 22002
Num. de registros en el dataframe del 2015: 23842
Num. de registros en el dataframe del 2016: 27534
Num. de registros en el dataframe del 2017: 27725
Núm. registros totales: 101103


### 2.1. Cuestionario del 2014

Primero, filtramos todos los registros que no pertenezcan a alumnos inscritos

In [4]:
df = dfPerfIn[2014].copy()

print('Num. registros totales', len(df))
print('Num. registros inscritos', len(df[df.numExpediente != 0 ]))
print('Num. registros NO inscritos', len(df[df.numExpediente == 0 ]))

df = df[df.numExpediente != 0 ]

Num. registros totales 22002
Num. registros inscritos 6562
Num. registros NO inscritos 15440


Se filtran las columnas que hacen referencia a los tutores y la vivienda del alumno (salvo la ciudad de origen)


In [5]:
df = df[[c for c in df.columns if 'tutor' not in c[-5:] and 'Tutor' not in c[-5:]]]
df = df.loc[:, (df.columns != 'entreCalles') & (df.columns != 'calleYNumero') & (df.columns != 'colonia') & (df.columns != 'cp')]

#### 2.1.1.Imputación de valores nulos

In [6]:
dfNanCount = df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1)
dfNanCount['prcntNans'] = dfNanCount.numNans*100/len(df)
dfNanCount[dfNanCount.numNans > 0]

,col,numNans,prcntNans
8,edoCivil,4,0.060957
10,ciudadOrigen,5,0.076196
20,razonTrabajo,4870,74.215178
21,horasTrabajoSemana,4870,74.215178
23,tieneCooler,4267,65.025907
24,tieneLavadora,837,12.755258
25,tieneTvDePaga,2610,39.774459
26,tieneAireAcondicionado,1928,29.381286
27,tieneAutoPropio,5817,88.646754
28,tieneAuto,1801,27.445901


In [7]:
df.replace(
    {'edoCivil':{np.nan:df.edoCivil.mode()},
     'ciudadOrigen':{np.nan:df.ciudadOrigen.mode()},
     'razonTrabajo':{np.nan:"No trabajo"}, 
     "horasTrabajoSemana":{np.nan:"No trabajo"},
     "tieneCooler":{np.nan:"no"},
     "tieneLavadora":{np.nan:"no"},
     "tieneTvDePaga":{np.nan:"no"},
     "tieneAireAcondicionado":{np.nan:"no"},
     "tieneAutoPropio":{np.nan:"no"},
     "tieneAuto":{np.nan:"no"},
     "tienePc":{np.nan:"no"},
     "tieneLaptop":{np.nan:"no"},
     "tieneInternet":{np.nan:"no"},
     "tieneEspacioParaEstudiar":{np.nan:"no"},
     "tieneCapacidadesParaEstudiarLaCarrera":{np.nan:df.tieneCapacidadesParaEstudiarLaCarrera.mode()},
     "satisfechoSiEjerceLaProfesion":{np.nan:df.satisfechoSiEjerceLaProfesion.mode()},
     "admiraPersonasQueEjercenLaProfesion":{np.nan:df.admiraPersonasQueEjercenLaProfesion.mode()},
     "estaMotivadoPorEstudiarLaCarrera":{np.nan:df.estaMotivadoPorEstudiarLaCarrera.mode()},
     "suAreaDeEstudioPrepaEsImportanteParaLaCarrera":{np.nan:df.suAreaDeEstudioPrepaEsImportanteParaLaCarrera.mode()},
     "conoceElPlanDeEstudios":{np.nan:df.conoceElPlanDeEstudios.mode()},
     "conoceOpcionesDeTrabajoDeLaCarrera":{np.nan:df.conoceOpcionesDeTrabajoDeLaCarrera.mode()},
     "consideraImportanteServiciosBibliotecarios":{np.nan:"no"},
     "consideraImportanteServiciosInformaticos":{np.nan:"no"},
     "consideraImportanteOrientacionPsicologica":{np.nan:"no"},
     "consideraImportanteOrientacionNutricional":{np.nan:"no"},
     "consideraImportanteConsultorioMedicoYDental":{np.nan:"no"},
     "consideraImportanteAsesoriaDePares":{np.nan:"no"},
     "consideraImportanteTutorias":{np.nan:"no"},
     "consideraImportanteActividadesDeportivas":{np.nan:"no"},
     "consideraImportanteActividadesCulturales":{np.nan:"no"},
     "consideraImportanteCursosDeIdiomas":{np.nan:"no"},
     "consideraImportanteMovilidadOIntercambio":{np.nan:"no"},
     "consideraImportanteBecas":{np.nan:"no"},
     "consideraImportanteTalleresSobreEstrategiasDeEstudio":{np.nan:"no"},
     "consideraImportanteTalleresDeHabilidadesSociales":{np.nan:"no"},
     "consideraImportanteCursosEnLinea":{np.nan:"no"},
     "consideraImportanteCursosParaRegularizarMateriasReprobadas":{np.nan:"no"},
     "consideraImportanteCulturaEmprendedora":{np.nan:"no"}
     }, 
     inplace=True
)

In [8]:
print("Num. nans en el Df:",df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1).numNans.sum())

Num. nans en el Df: 0


#### 2.1.2. Validación de la columna `empleado`

Se corregirán los valores de esta columna de acuerdo a las reglas:

IF `razonTrabajo` = 'No trabajo' AND `horasTrabajoSemana` = 'No trabajo' THEN `empleado` = No 

ELSE IF `razonTrabajo` = !'No trabajo' AND `horasTrabajoSemana` = !'No trabajo' THEN `empleado` = Si

Para lograr esta tarea, primero reemplazaremos los valores nulos de las columnas `razonTrabajo` y `horasTrabajoSemana` con la cadena "No trabajo"

Se verifica si existen inconsistencias

In [9]:
# Caso 1
print('Inconsistencia caso 1')
print(df[
    (df.empleado == 'no')&
    (df.razonTrabajo != 'No trabajo')&
    (df.horasTrabajoSemana != 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']])

# Caso 2
print('\n Inconsistencia caso 2')
print(df[
    (df.empleado == 'si')&
    (df.razonTrabajo == 'No trabajo')&
    (df.horasTrabajoSemana == 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']])

Inconsistencia caso 1
Empty DataFrame
Columns: [empleado, razonTrabajo, horasTrabajoSemana]
Index: []

 Inconsistencia caso 2
Empty DataFrame
Columns: [empleado, razonTrabajo, horasTrabajoSemana]
Index: []


#### 2.1.3. Validación del conteo de las preguntas del examen de admisión

In [10]:
def verificarSumasConteo(d, tipoCol):
    '''
        Esta función verifica si el conteo de preguntas clasificadas como
        aciertos/errores/no sé/no contestó del examen de admisión coinciden con
        la columna que reporta el total de éstas
        
        Params
        ------
        * d <pandas.DataFrame>: Dataframe en el que se pretende verificar los conteos
        * tipoCol <str>: Tipo de preguntas a verificar
        
        Returns
        -------
        * <bool>: Valor booleano que indica si la sumatoria es correcta
    '''
    sumatoria = d.loc[:, [c for c in d.columns if (c.endswith(tipoCol)) and c != f'total{tipoCol}']].sum(axis=1).astype('Int64')
    if tipoCol == 'Aciertos':
        real = d.totalAciertos.astype('Int64')
    elif tipoCol == 'Errores':
        real = d.totalErrores.astype('Int64')
    elif tipoCol == 'NoSe':
        real = d.totalNoSe.astype('Int64')
    elif tipoCol == 'NoContesto':
        real = d.totalNoContesto.astype('Int64')
    
    return sumatoria.equals(real)

In [11]:
for tipoConteo in ['Aciertos','Errores','NoSe','NoContesto']:
    print(f'sumatorias {tipoConteo} correcta?: ',verificarSumasConteo(df,tipoConteo))
print('-----------------------')

sumatorias Aciertos correcta?:  True
sumatorias Errores correcta?:  True
sumatorias NoSe correcta?:  True
sumatorias NoContesto correcta?:  True
-----------------------


Se actualiza el Dataframe

In [12]:
dfPerfIn[2014] = df

### 2.2. Cuestionario del 2015

Se filtran los registros que no pertenezcan a alumnos inscritos

In [13]:
df = dfPerfIn[2015].copy()

print('Num. registros totales', len(df))
print('Num. registros inscritos', len(df[df.numExpediente != 0 ]))
print('Num. registros NO inscritos', len(df[df.numExpediente == 0 ]))

df = df[df.numExpediente != 0 ]

Num. registros totales 23842
Num. registros inscritos 6616
Num. registros NO inscritos 17226


Se filtran las columnas que hacen referencia a los tutores y la vivienda del alumno (salvo la ciudad de origen)

In [14]:
df = df[[c for c in df.columns if 'tutor' not in c[-5:] and 'Tutor' not in c[-5:]]]
df = df.loc[:, (df.columns != 'entreCalles') & (df.columns != 'calleYNumero') & (df.columns != 'colonia') & (df.columns != 'cp')]

#### 2.2.1. Imputación de valores nulos

In [15]:
dfNanCount = df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1)
dfNanCount['prcntNans'] = dfNanCount.numNans*100/len(df)
dfNanCount[dfNanCount.numNans > 0]

,col,numNans,prcntNans
20,razonTrabajo,5040,76.178960
21,horasTrabajoSemana,5041,76.194075
23,tieneCooler,4486,67.805320
24,tieneLavadora,923,13.951028
25,tieneTvDePaga,2669,40.341596
26,tieneAireAcondicionado,1997,30.184401
27,tieneAutoPropio,5917,89.434704
28,tieneAuto,1923,29.065901
29,tienePc,3623,54.761185
30,tieneLaptop,2766,41.807739


In [16]:
df.replace(
    {
        'razonTrabajo':{np.nan:"No trabajo"},
        'horasTrabajoSemana':{np.nan:"No trabajo"},
        'tieneCooler':{np.nan:"no"},
        'tieneLavadora':{np.nan:"no"},
        'tieneTvDePaga':{np.nan:"no"},
        'tieneAireAcondicionado':{np.nan:"no"},
        'tieneAutoPropio':{np.nan:"no"},
        'tieneAuto':{np.nan:"no"},
        'tienePc':{np.nan:"no"},
        'tieneLaptop':{np.nan:"no"},
        'tieneInternet':{np.nan:"no"},
        'tieneEspacioParaEstudiar':{np.nan:"no"},
        'tieneCapacidadesParaEstudiarLaCarrera':{np.nan:df.tieneCapacidadesParaEstudiarLaCarrera.mode()},
        'satisfechoSiEjerceLaProfesion':{np.nan:df.satisfechoSiEjerceLaProfesion.mode()},
        'admiraPersonasQueEjercenLaProfesion':{np.nan:df.admiraPersonasQueEjercenLaProfesion.mode()},
        'estaMotivadoPorEstudiarLaCarrera':{np.nan:df.estaMotivadoPorEstudiarLaCarrera.mode()},
        'suAreaDeEstudioPrepaEsImportanteParaLaCarrera':{np.nan:df.suAreaDeEstudioPrepaEsImportanteParaLaCarrera.mode()},
        'conoceElPlanDeEstudios':{np.nan:df.conoceElPlanDeEstudios.mode()},
        'conoceOpcionesDeTrabajoDeLaCarrera':{np.nan:df.conoceOpcionesDeTrabajoDeLaCarrera.mode()},
        'consideraImportanteServiciosBibliotecarios':{np.nan:"no"},
        'consideraImportanteServiciosInformaticos':{np.nan:"no"},
        'consideraImportanteOrientacionPsicologica':{np.nan:"no"},
        'consideraImportanteOrientacionNutricional':{np.nan:"no"},
        'consideraImportanteConsultorioMedicoYDental':{np.nan:"no"},
        'consideraImportanteAsesoriaDePares':{np.nan:"no"},
        'consideraImportanteTutorias':{np.nan:"no"},
        'consideraImportanteActividadesDeportivas':{np.nan:"no"},
        'consideraImportanteActividadesCulturales':{np.nan:"no"},
        'consideraImportanteCursosDeIdiomas':{np.nan:"no"},
        'consideraImportanteMovilidadOIntercambio':{np.nan:"no"},
        'consideraImportanteBecas':{np.nan:"no"},
        'consideraImportanteTalleresSobreEstrategiasDeEstudio':{np.nan:"no"},
        'consideraImportanteTalleresDeHabilidadesSociales':{np.nan:"no"},
        'consideraImportanteCursosEnLinea':{np.nan:"no"},
        'consideraImportanteCursosParaRegularizarMateriasReprobadas':{np.nan:"no"},
        'consideraImportanteCulturaEmprendedora':{np.nan:"no"}
    }, 
    inplace=True
)

In [17]:
print("Num. nans en el Df:",df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1).numNans.sum())

Num. nans en el Df: 0


#### 2.2.2. Validación de la columna `empleado`

In [18]:
# Caso 1
print('Inconsistencia caso 1')
print(df[
    (df.empleado == 'no')&
    (df.razonTrabajo != 'No trabajo')&
    (df.horasTrabajoSemana != 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']])

# Caso 2
print('\n Inconsistencia caso 2')
print(df[
    (df.empleado == 'si')&
    (df.razonTrabajo == 'No trabajo')&
    (df.horasTrabajoSemana == 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']])

Inconsistencia caso 1
Empty DataFrame
Columns: [empleado, razonTrabajo, horasTrabajoSemana]
Index: []

 Inconsistencia caso 2
Empty DataFrame
Columns: [empleado, razonTrabajo, horasTrabajoSemana]
Index: []


#### 2.2.3. Validación del conteo de las preguntas del examen de admisión

In [19]:
for tipoConteo in ['Aciertos','Errores','NoSe','NoContesto']:
    print(f'sumatorias {tipoConteo} correcta?: ',verificarSumasConteo(df,tipoConteo))
print('-----------------------')

sumatorias Aciertos correcta?:  True
sumatorias Errores correcta?:  True
sumatorias NoSe correcta?:  True
sumatorias NoContesto correcta?:  True
-----------------------


Se actualiza el Dataframe

In [20]:
dfPerfIn[2015] = df

### 2.3. Cuestionario del 2016

Se filtran los registros que no pertenezcan a alumnos inscritos

In [21]:
df = dfPerfIn[2016].copy()

print('Num. registros totales', len(df))
print('Num. registros inscritos', len(df[df.numExpediente != 0 ]))
print('Num. registros NO inscritos', len(df[df.numExpediente == 0 ]))

df = df[df.numExpediente != 0 ]

Num. registros totales 27534
Num. registros inscritos 7187
Num. registros NO inscritos 20347


Se filtran las columnas que hacen referencia a los tutores y la vivienda del alumno (salvo la ciudad de origen)

In [22]:
df = df[[c for c in df.columns if 'tutor' not in c[-5:] and 'Tutor' not in c[-5:]]]
df = df.loc[:, (df.columns != 'entreCalles') & (df.columns != 'calleYNumero') & (df.columns != 'colonia') & (df.columns != 'cp')]

#### 2.3.1. Imputación de valores nulos

In [23]:
dfNanCount = df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1)
dfNanCount['prcntNans'] = dfNanCount.numNans*100/len(df)
dfNanCount[dfNanCount.numNans > 0]

,col,numNans,prcntNans
12,modalidadEstudioPrepa,2,0.027828
13,nombrePrepa,1,0.013914
23,tieneCooler,5058,70.377070
24,tieneTvDePaga,2607,36.273828
25,tieneAireAcondicionado,1964,27.327118
26,tieneAutoPropio,6576,91.498539
27,tieneAuto,2145,29.845554
28,tienePc,4204,58.494504
29,tieneLaptop,3035,42.229025
30,tieneInternet,1356,18.867399


In [24]:
df.replace(
    {
        'modalidadEstudioPrepa':{np.nan:df.modalidadEstudioPrepa.mode()},
        'nombrePrepa':{np.nan:df.nombrePrepa.mode()},
        'tieneCooler':{np.nan:"no"},
        'tieneTvDePaga':{np.nan:"no"},
        'tieneAireAcondicionado':{np.nan:"no"},
        'tieneAutoPropio':{np.nan:"no"},
        'tieneAuto':{np.nan:"no"},
        'tienePc':{np.nan:"no"},
        'tieneLaptop':{np.nan:"no"},
        'tieneInternet':{np.nan:"no"},
        'tieneEspacioParaEstudiar':{np.nan:"no"},
        'duermeTiempoSuficiente':{np.nan:"no"},
        'haceEjercicioFisico':{np.nan:"no"},
        'desayuna':{np.nan:"no"},
        'comeYCenaEnHorariosRegulares':{np.nan:"no"},
        'consumeAlcohol':{np.nan:"no"},
        'consumeTabaco':{np.nan:"no"},
        'conformeConModoDeSer':{np.nan:"no"},
        'conformeConCapacidadIntelectual':{np.nan:"no"},
        'conformeCapacidadRelacionarse':{np.nan:"no"},
        'conformeCapacidadEnfrentarProblemas':{np.nan:"no"},
        'conformeCapacidadAdaptarseACambios':{np.nan:"no"},
        'expresaCariño':{np.nan:"no"},
        'solicitaAyuda':{np.nan:"no"},
        'expresaOpiniones':{np.nan:"no"},
        'exponeDudasEnLaEscuela':{np.nan:"no"},
        'hablaEnPublico':{np.nan:"no"},
        'resistePresionSocial':{np.nan:"no"},
        'exponeEnfadoSinAgredir':{np.nan:"no"},
        'aceptaCriticas':{np.nan:"no"},
        'estableceAmistadesDeCalidad':{np.nan:"no"},
        'enfrentaDificultadesEscolares':{np.nan:"no"},
        'estudiaRepitiendoLoQueHayQueAprender':{np.nan:"no"},
        'estudiaCopiandoLoMismoDelLibro':{np.nan:"no"},
        'estudiaSubrayandoLoImportante':{np.nan:"no"},
        'estudiaEscribiendoConPalabrasPropiasLoAprendido':{np.nan:"no"},
        'estudiaImaginandoLoQueHayQueAprender':{np.nan:"no"},
        'estudiaTomandoNotasEnClase':{np.nan:"no"},
        'estudiaHaciendoResumenOSintesis':{np.nan:"no"},
        'estudiaRelacionandoLoNuevoConLoConocido':{np.nan:"no"},
        'estudiaEncontrandoOrdenYClasificando':{np.nan:"no"},
        'estudiaElaborandoMapasConceptuales':{np.nan:"no"},
        'prefiereExamenesDeOpcionMultiple':{np.nan:"no"},
        'prefiereCompletarDatos':{np.nan:"no"},
        'prefiereSintetizar':{np.nan:"no"},
        'prefiereSolucionarProblemas':{np.nan:"no"},
        'prefiereExpresarYJustificarIdeas':{np.nan:"no"},
        'prefiereHacerProyectos':{np.nan:"no"},
        'prefiereHorarioRegularDeEstudio':{np.nan:"no"},
        'prefiereLlevarAgendaDePendientes':{np.nan:"no"},
        'prefiereConseguirMaterialesATiempo':{np.nan:"no"},
        'prefiereRepasarDiariamente':{np.nan:"no"},
        'prefiereParticiparEnClase':{np.nan:"no"},
        'prefiereBuscarAsesoria':{np.nan:"no"},
        'prefiereEstudiarEnEquipo':{np.nan:"no"},
        'tieneCapacidadesParaEstudiarLaCarrera':{np.nan:df.tieneCapacidadesParaEstudiarLaCarrera.mode()},
        'satisfechoSiEjerceLaProfesion':{np.nan:df.satisfechoSiEjerceLaProfesion.mode()},
        'admiraPersonasQueEjercenLaProfesion':{np.nan:df.admiraPersonasQueEjercenLaProfesion.mode()},
        'estaMotivadoPorEstudiarLaCarrera':{np.nan:df.estaMotivadoPorEstudiarLaCarrera.mode()},
        'suAreaDeEstudioPrepaEsImportanteParaLaCarrera':{np.nan:df.suAreaDeEstudioPrepaEsImportanteParaLaCarrera.mode()},
        'conoceElPlanDeEstudios':{np.nan:df.conoceElPlanDeEstudios.mode()},
        'conoceOpcionesDeTrabajoDeLaCarrera':{np.nan:df.conoceOpcionesDeTrabajoDeLaCarrera.mode()},
        'elPrestigioDeLaUnisonHaInfluidoEnSuEleccionDeUniversidad':{np.nan:df.elPrestigioDeLaUnisonHaInfluidoEnSuEleccionDeUniversidad.mode()},
        'seDecidioPorLaUnisonPorSuExcelenteCalidad':{np.nan:df.seDecidioPorLaUnisonPorSuExcelenteCalidad.mode()},
        'laUbicacionDelCampusInfluyeEnSuEleccion':{np.nan:df.laUbicacionDelCampusInfluyeEnSuEleccion.mode()},
        'susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica':{np.nan:df.susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica.mode()},
        'laExigenciaSeraSimilarALaPrepa':{np.nan:df.laExigenciaSeraSimilarALaPrepa.mode()},
        'leSeraFacilIntegrarseYTrabajarEnEquipo':{np.nan:df.leSeraFacilIntegrarseYTrabajarEnEquipo.mode()},
        'tieneClarasLasExigenciasDeUnaCarreraUniversitaria':{np.nan:df.tieneClarasLasExigenciasDeUnaCarreraUniversitaria.mode()},
        'laCarreraLePermitiraMejorarSuSituacionActual':{np.nan:df.laCarreraLePermitiraMejorarSuSituacionActual.mode()},
        'concluiraEnElTiempoPrevisto':{np.nan:df.concluiraEnElTiempoPrevisto.mode()},
        'piensaEstudiarUnPosgradoAlConcluir':{np.nan:df.piensaEstudiarUnPosgradoAlConcluir.mode()}
    }, 
    inplace=True
)

In [25]:
print("Num. nans en el Df:",df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1).numNans.sum())

Num. nans en el Df: 0


#### 2.3.2. Validación de la columna `empleado`

In [26]:
# Caso 1
print('Inconsistencia caso 1')
df[
    (df.empleado == 0)&
    (df.razonTrabajo != 'No trabajo')&
    (df.horasTrabajoSemana != 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']]

Inconsistencia caso 1


,empleado,razonTrabajo,horasTrabajoSemana
12,0,Gastos personales,Menos de 10 horas
21,0,Sostener mis estudios,Menos de 10 horas
37,0,Gastos personales,Menos de 10 horas
71,0,Gastos personales,Menos de 10 horas
98,0,Gastos personales,Menos de 10 horas
...,...,...,...
27453,0,Gastos personales,Menos de 10 horas
27454,0,Sostener mis estudios,Menos de 10 horas
27506,0,Sostener mis estudios,Menos de 10 horas
27512,0,Sostener mis estudios,Menos de 10 horas


In [27]:
# Caso 2
print('\n Inconsistencia caso 2')
df[
    (df.empleado == 1)&
    (df.razonTrabajo == 'No trabajo')&
    (df.horasTrabajoSemana == 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']]


 Inconsistencia caso 2


,empleado,razonTrabajo,horasTrabajoSemana


Existen inconsistencias en esta columna. Hay casos en donde se indica que el estudiante no es empleado, sin embargo reporta razones por las cuales trabaja y las horas que trabaja. Se corregirán estos registros

In [28]:
df.horasTrabajoSemana.unique()

array(['Menos de 10 horas'], dtype=object)

In [29]:
df.loc[~df.razonTrabajo.str.contains('No trabajo')&~df.horasTrabajoSemana.str.contains('No trabajo'), 'empleado'] = 1

Se vuelve a ejecutar la validación

In [30]:
# Caso 1
print('Inconsistencia caso 1')
df[
    (df.empleado == 0)&
    (df.razonTrabajo != 'No trabajo')&
    (df.horasTrabajoSemana != 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']]

Inconsistencia caso 1


,empleado,razonTrabajo,horasTrabajoSemana


In [31]:
# Caso 2
print('\n Inconsistencia caso 2')
df[
    (df.empleado == 1)&
    (df.razonTrabajo == 'No trabajo')&
    (df.horasTrabajoSemana == 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']]


 Inconsistencia caso 2


,empleado,razonTrabajo,horasTrabajoSemana


#### 2.3.3. Validación del conteo de las preguntas del examen de admisión

In [32]:
for tipoConteo in ['Aciertos','Errores','NoSe','NoContesto']:
    print(f'sumatorias {tipoConteo} correcta?: ',verificarSumasConteo(df,tipoConteo))
print('-----------------------')

sumatorias Aciertos correcta?:  True
sumatorias Errores correcta?:  True
sumatorias NoSe correcta?:  True
sumatorias NoContesto correcta?:  True
-----------------------


Se actualiza el dataframe

In [33]:
# Se corrige el nombre de una columna
df = df.rename(columns={'susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica':'susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica'})
dfPerfIn[2016] = df

### 2.4. Cuestionario del 2017

Se filtran los registros que no pertenezcan a alumnos inscritos

In [34]:
df = dfPerfIn[2017].copy()

print('Num. registros totales', len(df))
print('Num. registros inscritos', len(df[df.numExpediente.notna() ]))
print('Num. registros NO inscritos', len(df[df.numExpediente.isna() ]))

df = df[df.numExpediente.notna() ]

Num. registros totales 27725
Num. registros inscritos 7146
Num. registros NO inscritos 20579


Se filtran las columnas que hacen referencia a los tutores y la vivienda del alumno (salvo la ciudad de origen)

In [35]:
df = df[[c for c in df.columns if 'tutor' not in c[-5:] and 'Tutor' not in c[-5:]]]
df = df.loc[:, (df.columns != 'entreCalles') & (df.columns != 'calleYNumero') & (df.columns != 'colonia') & (df.columns != 'cp')]

#### 2.4.1. Imputación de valores nulos

In [36]:
dfNanCount = df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1)
dfNanCount['prcntNans'] = dfNanCount.numNans*100/len(df)
dfNanCount[dfNanCount.numNans > 0]

,col,numNans,prcntNans
13,nombrePrepa,1,0.013994
22,tieneCooler,5167,72.306185
23,tieneLavadora,966,13.518052
24,tieneTvDePaga,2777,38.860901
25,tieneAireAcondicionado,1835,25.678701
26,tieneAutoPropio,6553,91.701651
27,tieneAuto,2052,28.715365
28,tienePc,4292,60.061573
29,tieneLaptop,2803,39.224741
30,tieneInternet,1089,15.239295


In [37]:
[c for c in df.columns]

['idRegistroAspirante',
 'idSolicitudAspirante',
 'numExpediente',
 'fechaNacimiento',
 'lugarPreferencia',
 'campus',
 'nacionalidad',
 'sexo',
 'edoCivil',
 'gpoIndigena',
 'ciudadOrigen',
 'sistemaPrepa',
 'modalidadEstudioPrepa',
 'nombrePrepa',
 'ciudadPrepa',
 'areaEstudioPrepa',
 'anioIngresoPrepa',
 'anioEgresoPrepa',
 'sostenEconomico',
 'situacionEconFav',
 'razonTrabajo',
 'horasTrabajoSemana',
 'tieneCooler',
 'tieneLavadora',
 'tieneTvDePaga',
 'tieneAireAcondicionado',
 'tieneAutoPropio',
 'tieneAuto',
 'tienePc',
 'tieneLaptop',
 'tieneInternet',
 'tieneEspacioParaEstudiar',
 'situacionPadres',
 'situacionFamilia',
 'consecuenciaDeSituacionEconomica',
 'discapacidadParalisis',
 'discapacidadFaltaExtremidades',
 'discapacidadSordera',
 'discapacidadCeguera',
 'discapacidadDebilidadVisual',
 'discapacidadDebilidadAuditiva',
 'discapacidadOtra',
 'duermeTiempoSuficiente',
 'haceEjercicioFisico',
 'desayuna',
 'comeYCenaEnHorariosRegulares',
 'consumeAlcohol',
 'consumeTabac

In [38]:
df.replace(
    {
        'nombrePrepa':{np.nan:df.nombrePrepa.mode()},
        'tieneCooler':{np.nan:"no"},
        'tieneLavadora':{np.nan:"no"},
        'tieneTvDePaga':{np.nan:"no"},
        'tieneAireAcondicionado':{np.nan:"no"},
        'tieneAutoPropio':{np.nan:"no"},
        'tieneAuto':{np.nan:"no"},
        'tienePc':{np.nan:"no"},
        'tieneLaptop':{np.nan:"no"},
        'tieneInternet':{np.nan:"no"},
        'tieneEspacioParaEstudiar':{np.nan:"no"},
        'duermeTiempoSuficiente':{np.nan:"no"},
        'haceEjercicioFisico':{np.nan:"no"},
        'desayuna':{np.nan:"no"},
        'comeYCenaEnHorariosRegulares':{np.nan:"no"},
        'consumeAlcohol':{np.nan:"no"},
        'consumeTabaco':{np.nan:"no"},
        'conformeConModoDeSer':{np.nan:"no"},
        'conformeConCapacidadIntelectual':{np.nan:"no"},
        'conformeCapacidadRelacionarse':{np.nan:"no"},
        'conformeCapacidadEnfrentarProblemas':{np.nan:"no"},
        'conformeCapacidadAdaptarseACambios':{np.nan:"no"},
        'expresaCariño':{np.nan:"no"},
        'solicitaAyuda':{np.nan:"no"},
        'expresaOpiniones':{np.nan:"no"},
        'exponeDudasEnLaEscuela':{np.nan:"no"},
        'hablaEnPublico':{np.nan:"no"},
        'resistePresionSocial':{np.nan:"no"},
        'exponeEnfadoSinAgredir':{np.nan:"no"},
        'aceptaCriticas':{np.nan:"no"},
        'estableceAmistadesDeCalidad':{np.nan:"no"},
        'enfrentaDificultadesEscolares':{np.nan:"no"},
        'estudiaRepitiendoLoQueHayQueAprender':{np.nan:"no"},
        'estudiaCopiandoLoMismoDelLibro':{np.nan:"no"},
        'estudiaSubrayandoLoImportante':{np.nan:"no"},
        'estudiaEscribiendoConPalabrasPropiasLoAprendido':{np.nan:"no"},
        'estudiaImaginandoLoQueHayQueAprender':{np.nan:"no"},
        'estudiaTomandoNotasEnClase':{np.nan:"no"},
        'estudiaHaciendoResumenOSintesis':{np.nan:"no"},
        'estudiaRelacionandoLoNuevoConLoConocido':{np.nan:"no"},
        'estudiaEncontrandoOrdenYClasificando':{np.nan:"no"},
        'estudiaElaborandoMapasConceptuales':{np.nan:"no"},
        'prefiereExamenesDeOpcionMultiple':{np.nan:"no"},
        'prefiereCompletarDatos':{np.nan:"no"},
        'prefiereSintetizar':{np.nan:"no"},
        'prefiereSolucionarProblemas':{np.nan:"no"},
        'prefiereExpresarYJustificarIdeas':{np.nan:"no"},
        'prefiereHacerProyectos':{np.nan:"no"},
        'prefiereHorarioRegularDeEstudio':{np.nan:"no"},
        'prefiereLlevarAgendaDePendientes':{np.nan:"no"},
        'prefiereConseguirMaterialesATiempo':{np.nan:"no"},
        'prefiereRepasarDiariamente':{np.nan:"no"},
        'prefiereParticiparEnClase':{np.nan:"no"},
        'prefiereBuscarAsesoria':{np.nan:"no"},
        'prefiereEstudiarEnEquipo':{np.nan:"no"},
        'tieneCapacidadesParaEstudiarLaCarrera':{np.nan:df.tieneCapacidadesParaEstudiarLaCarrera.mode()},
        'satisfechoSiEjerceLaProfesion':{np.nan:df.satisfechoSiEjerceLaProfesion.mode()},
        'admiraPersonasQueEjercenLaProfesion':{np.nan:df.admiraPersonasQueEjercenLaProfesion.mode()},
        'estaMotivadoPorEstudiarLaCarrera':{np.nan:df.estaMotivadoPorEstudiarLaCarrera.mode()},
        'suAreaDeEstudioPrepaEsImportanteParaLaCarrera':{np.nan:df.suAreaDeEstudioPrepaEsImportanteParaLaCarrera.mode()},
        'conoceElPlanDeEstudios':{np.nan:df.conoceElPlanDeEstudios.mode()},
        'conoceOpcionesDeTrabajoDeLaCarrera':{np.nan:df.conoceOpcionesDeTrabajoDeLaCarrera.mode()},
        'elPrestigioDeLaUnisonHaInfluidoEnSuEleccionDeUniversidad':{np.nan:df.elPrestigioDeLaUnisonHaInfluidoEnSuEleccionDeUniversidad.mode()},
        'seDecidioPorLaUnisonPorSuExcelenteCalidad':{np.nan:df.seDecidioPorLaUnisonPorSuExcelenteCalidad.mode()},
        'laUbicacionDelCampusInfluyeEnSuEleccion':{np.nan:df.laUbicacionDelCampusInfluyeEnSuEleccion.mode()},
        'susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica':{np.nan:df.susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica.mode()},
        'laExigenciaSeraSimilarALaPrepa':{np.nan:df.laExigenciaSeraSimilarALaPrepa.mode()},
        'leSeraFacilIntegrarseYTrabajarEnEquipo':{np.nan:df.leSeraFacilIntegrarseYTrabajarEnEquipo.mode()},
        'tieneClarasLasExigenciasDeUnaCarreraUniversitaria':{np.nan:df.tieneClarasLasExigenciasDeUnaCarreraUniversitaria.mode()},
        'laCarreraLePermitiraMejorarSuSituacionActual':{np.nan:df.laCarreraLePermitiraMejorarSuSituacionActual.mode()},
        'concluiraEnElTiempoPrevisto':{np.nan:df.concluiraEnElTiempoPrevisto.mode()},
        'piensaEstudiarUnPosgradoAlConcluir':{np.nan:df.piensaEstudiarUnPosgradoAlConcluir.mode()}
    }, 
    inplace=True
)

In [39]:
print("Num. nans en el Df:",df.isna().sum().to_frame().reset_index().rename({'index':'col', 0:'numNans'}, axis=1).numNans.sum())

Num. nans en el Df: 0


#### 2.4.2. Creación y validación de la columna `empleado`

In [40]:
df['empleado'] = np.nan
df.loc[df.razonTrabajo.str.contains('No trabajo')&df.horasTrabajoSemana.str.contains('No trabajo'), 'empleado'] = 'No'
df.loc[~df.razonTrabajo.str.contains('No trabajo')&~df.horasTrabajoSemana.str.contains('No trabajo'), 'empleado'] = 'Si'

# Caso 1
print('Inconsistencia caso 1')
print(df[
    (df.empleado == 0)&
    (df.razonTrabajo != 'No trabajo')&
    (df.horasTrabajoSemana != 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']])

# Caso 2
print('\n Inconsistencia caso 2')
print(df[
    (df.empleado == 1)&
    (df.razonTrabajo == 'No trabajo')&
    (df.horasTrabajoSemana == 'No trabajo')
][['empleado', 'razonTrabajo', 'horasTrabajoSemana']])

Inconsistencia caso 1
Empty DataFrame
Columns: [empleado, razonTrabajo, horasTrabajoSemana]
Index: []

 Inconsistencia caso 2
Empty DataFrame
Columns: [empleado, razonTrabajo, horasTrabajoSemana]
Index: []


In [41]:
df.loc[df.empleado.isna(), ['razonTrabajo', 'horasTrabajoSemana', 'empleado']].head(10)

,razonTrabajo,horasTrabajoSemana,empleado
26,Ayudar al gasto familiar,No trabajo,NaN
103,Ayudar al gasto familiar,No trabajo,NaN
124,No trabajo,Menos de 10 horas,NaN
581,Ayudar al gasto familiar,No trabajo,NaN
1298,Sostener mis estudios,No trabajo,NaN
1427,Adquirir experiencia laboral,No trabajo,NaN
1512,Sostener mis estudios,No trabajo,NaN
1543,Ayudar al gasto familiar,No trabajo,NaN
1577,Adquirir experiencia laboral,No trabajo,NaN
1578,Ayudar al gasto familiar,No trabajo,NaN


In [42]:
print('renglones con la columna "empleado" inconclusa:', len(df[df.empleado.isna()]), '(',len(df[df.empleado.isna()])/len(df)*100,'%)')


renglones con la columna "empleado" inconclusa: 130 ( 1.8191995521970334 %)


Como se puede observar, existen inconsistencias en las respuestas de los aspirantes: Algunos de ellos tienen una razon para trabajar, pero al mismo tiempo trabajan 0 horas a la semana; otros, no tienen razón para trabajar, pero reportan horas trabajadas a la semana. Para todos estos casos concluiremos que no son trabajadores.

In [43]:
df.empleado = df.empleado.fillna('No')
print('renglones con la columna "empleado" inconclusa:', len(df[df.empleado.isna()]), '(',len(df[df.empleado.isna()])/len(df)*100,'%)')

renglones con la columna "empleado" inconclusa: 0 ( 0.0 %)


#### 2.4.3. Validación del conteo de las preguntas del examen de admisión

In [44]:
for tipoConteo in ['Aciertos','Errores','NoSe','NoContesto']:
    print(f'sumatorias {tipoConteo} correcta?: ',verificarSumasConteo(df,tipoConteo))
print('-----------------------')

sumatorias Aciertos correcta?:  True
sumatorias Errores correcta?:  True
sumatorias NoSe correcta?:  True
sumatorias NoContesto correcta?:  True
-----------------------


Se actualiza el dataframe

In [45]:
# Se corrige el nombre de una columna
df = df.rename(columns={'susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica':'susRecursosEconomicosSoloLePermitenEstudiarEnUnaUniversidadPublica'})
dfPerfIn[2017] = df

## Almacenamiento de los datos

Finalmente, almacenamos los dataframes actualizados

In [46]:
path = '../correctedDfs/perfilIngreso/'
if not os.path.exists(path):
    os.makedirs(path)

for anio in dfPerfIn.keys():
    dfPerfIn[anio].to_pickle(f'{path}{anio}.pkl')